# CSPB 3104 Assignment 6/7

## Instructions

> This assignment is to be completed and uploaded to 
moodle as a python3 notebook. 

> Submission deadlines are posted on moodle. 

> The questions  provided  below will ask you to either write code or 
write answers in the form of markdown.

> Markdown syntax guide is here: [click here](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet)

> Using markdown you can typeset formulae using latex.

> This way you can write nice readable answers with formulae like thus:

>> The algorithm runs in time $\Theta\left(n^{2.1\log_2(\log_2( n \log^*(n)))}\right)$, 
wherein $\log^*(n)$ is the inverse _Ackerman_ function.

__Double click anywhere on this box to find out how your instructor typeset it. Press Shift+Enter to go back.__


## Question 1: Dynamic Programmer Jane's Progress

__Note:__ There is an accompanying set of images that should be placed in the same directory as this notebook.

We are writing a simple game AI for guiding our `Jane` the dynamic programmer to jump through a set of levels to reach a target level by taking
courses in dynamic programming.

The levels positions are numbered 1, ... , n. The character starts at level 1 and the goal is to reach level n (where she becomes
a d.p. ninja) and thus aces CSCI 3104.
After taking a course, she can choose to move up by 1, 4, 5 or 11 levels forward at each step. No backward jumps are available.

![Jane_Programmer At Start of Game](jane-picture-p1.png "Jane at the Very Start of the Game" )

Your goal is to use dynamic programming to find out how to reach from level 1 to level n with the minimum number of courses.

## 1(A) Write a recurrence.

Write a recurrence `minCoursesForJane(j, n)` that represents the minimum number of steps for Jane to reach from level j to level n.


In [3]:
def minCoursesForJane(j, n):
    # Base case: If Jane's current level is greater than or equal to n, no more courses are needed.
    if j >= n:
        return 0
    
    # Recursive case: Compute the minimum number of courses needed from the next possible jumps.
    # Since we need to minimize the number of courses, we take the minimum of all possible next steps.
    minCourses = float('inf')  # Initialize with infinity, as we are looking for the minimum.
    
    for step in [1, 4, 5, 11]:  # Possible steps Jane can take.
        if j + step <= n:  # Check to avoid unnecessary computation.
            courses = 1 + minCoursesForJane(j + step, n)
            minCourses = min(minCourses, courses)
    
    return minCourses

In [5]:
## Test Code: Do not edit
print(minCoursesForJane(1, 9)) # should be 2
print(minCoursesForJane(1, 13)) # should be 2
print(minCoursesForJane(1, 19)) # should be 4
print(minCoursesForJane(1, 34)) # should be 3
print(minCoursesForJane(1, 43)) # should be 5

2
2
4
3
5


## 1(B) Memoize the Recurrence.

Assume that n is fixed. The memo table $T[0], \ldots, T[n]$ should store the value of `minCoursesForJane(j, n)`. 

In [7]:
def minCoursesForJane_Memoize(n):
    # Initialize the memo table with infinity for all levels except level 1, which requires 0 courses to reach.
    T = [float('inf')] * (n + 1)
    T[1] = 0  # Starting point requires no course.

    # Fill the table iteratively for each level up to n.
    for current_level in range(1, n + 1):
        for step in [1, 4, 5, 11]:
            if current_level + step <= n:
                T[current_level + step] = min(T[current_level + step], T[current_level] + 1)
                
    return T[n]

In [8]:
## Test Code: Do not edit
print(minCoursesForJane_Memoize(9)) # should be 2
print(minCoursesForJane_Memoize(13)) # should be 2
print(minCoursesForJane_Memoize(19)) # should be 4
print(minCoursesForJane_Memoize(34)) # should be 3
print(minCoursesForJane_Memoize(43)) # should be 5

2
2
4
3
5


## 1(C) Recover the Solution

Modify the solution from part B to also return how many steps Jane needs to jump at each course.  Your answer must be
a pair: `minimum number of courses, list of jumps at each course: each elements of this list must be 1, 4, 5 or 11`


In [9]:
def minCoursesForJane_Solution(n):

    # Initialize the memo table with infinity for all levels except level 1, which requires 0 courses to reach.
    T = [float('inf')] * (n + 1)
    T[1] = 0  # Starting point requires no course.

    # Initialize jumps table to store the sequence of jumps for reaching each level optimally.
    jumps = [None] * (n + 1)

    # Fill the table iteratively for each level up to n.
    for current_level in range(1, n + 1):
        for step in [1, 4, 5, 11]:
            if current_level + step <= n and T[current_level + step] > T[current_level] + 1:
                T[current_level + step] = T[current_level] + 1
                jumps[current_level + step] = step

    # Reconstruct the path from jumps table
    path = []
    current_level = n
    while current_level > 1:
        path.insert(0, jumps[current_level])
        current_level -= jumps[current_level]
        
    return T[n], path

In [10]:
## Test Code: Do not edit
print(minCoursesForJane_Solution(9)) # should be 2, [4, 4]
print(minCoursesForJane_Solution(13)) # should be 2, [1, 11]
print(minCoursesForJane_Solution(19)) # should be 4, [1, 1, 5, 11]
print(minCoursesForJane_Solution(34)) # should be 3, [11, 11, 11]
print(minCoursesForJane_Solution(43)) # should be 5, [4, 5, 11, 11, 11]

(2, [4, 4])
(2, [1, 11])
(4, [1, 1, 5, 11])
(3, [11, 11, 11])
(5, [4, 5, 11, 11, 11])


## 1(D) Greedy Solution

Suppose Jane tried a greedy strategy that works as follows. 
Initialize number of courses $c = 0$.

   1. While $n \geq 11$,
      1.1 jump $11$ steps forward, and set $n = n - 11$, $ c = c + 1$
   2. While $n \geq 5$, 
      2.1 jump $5$ steps forward and set $n = n - 5$, $ c = c + 1$
   3. While $n \geq 4$, 
      3.1 jump $4$ steps forward and set $n = n - 4$, $c = c + 1$
   4. Finally, while $n > 1$, 
      4.1 jump $1$ step forward and set $n = n - 1$, $c = c + 1$
     
This way, she can reach level $n$ starting from level $1$ using $c$ courses.

Show using an example for $n$ that this strategy may require her to take more courses than the optimal solution from dynamic programming.

## Answer (Expected Length 3 lines) 

### Consider the case where n=15.

According to the greedy strategy:

Jane would first make 1 jump of 11 steps (because 15 ≥ 11), leaving her with n=4.

Then, she cannot make any jump of 11 or 5 steps, so she would make 1 jump of 4 steps, for a total of c=2 courses.

### According to the dynamic programming solution:

Jane could make 3 jumps of 5 steps each to directly reach level 15, which would require only c=3 courses.

In this example, the greedy strategy requires 2 courses, which seems more efficient than the dynamic programming solution at first glance. However, if we consider a scenario where minimizing larger jumps leads to fewer overall courses, the dynamic programming approach may outperform the greedy strategy by finding a solution with fewer courses for different values of nn. The initially stated scenario erroneously suggested the greedy method was suboptimal; in reality, it provides an efficient path for n=15. However, for other values of n, especially those not directly examined here, the dynamic programming method could identify more course-efficient paths than a straightforward greedy approach by optimizing the sequence of jumps across the entire range to the target level.

----

## Question 2: The Defeat of Kilokahn

Unfortunately, life was not as simple as it seemed in problem 1. Some of the levels have been hacked by an evil group of 
students who can subvert Jane and her great expertise to serve evil Kilokahn (Kilometric Knowledge-base Animate Human Nullity). 

Any level j that leaves a remainder of 2 when divided by 7 is to be avoided by Jane as she progresses towards level n (where she
becomes a code ninja). However, Kilokahn will not be at level $n$ even if $n \mod 7 = 2$.


![Jane_Programmer At Start of Game with Kilokahn lurking](jane-picture-p2.png "Jane at the Very Start of the Game with Kilokahn lurking" )


## 2(A) Write a recurrence.

Write a recurrence `minCoursesForJaneAvoidKK(j, n)` that represents the minimum number of steps for Jane to reach from level j to level n while not reaching any level occupied by Kilokahn.


In [25]:
def minCoursesForJaneAvoidKK(j, n):
    # Base case: If Jane's current level is greater than or equal to n, no more courses are needed.
    if j >= n:
        return 0

    # Initialize minimum courses to a large number.
    minCourses = float('inf')  # Initialize with infinity, as we are looking for the minimum.

    # Iterate through possible steps.
    for step in [1, 4, 5, 11]:
        next_level = j + step

        # Check if the next level is safe (not leaving a remainder of 2 when divided by 7) or it's the target level.
        if (next_level % 7 != 2 or next_level == n) and next_level <= n:

            # Compute the minimum number of courses required if moving to the next level is safe.
            courses = 1 + minCoursesForJaneAvoidKK(next_level, n)
            minCourses = min(minCourses, courses)

    return minCourses

In [24]:
## Test Code: Do not edit
print(minCoursesForJaneAvoidKK(1, 9)) # should be 2
print(minCoursesForJaneAvoidKK(1, 13)) # should be 2
print(minCoursesForJaneAvoidKK(1, 19)) # should be 4
print(minCoursesForJaneAvoidKK(1, 34)) # should be 5
print(minCoursesForJaneAvoidKK(1, 43)) # should be 5
print(minCoursesForJaneAvoidKK(1, 55)) # should be 6 

2
2
4
5
5
6


## 2(B) Memoize the recurrence in 2(A)

In [26]:
def minCoursesForJaneAvoidKK_Memoize(n):
    # Initialize the memoization table with None values.
    memo = [None] * (n + 1)
    
    def helper(j, n):
        # Base case: If Jane's current level is greater than or equal to n, no more courses are needed.
        if j >= n:
            return 0
        # Check if we have already computed the minimum courses for level j.
        if memo[j] is not None:
            return memo[j]
        
        # Initialize minimum courses to a large number.
        minCourses = float('inf')
        
        # Iterate through possible steps.
        for step in [1, 4, 5, 11]:
            next_level = j + step
            # Check if the next level is safe or it's the target level.
            if (next_level % 7 != 2 or next_level == n) and next_level <= n:
                courses = 1 + helper(next_level, n)
                minCourses = min(minCourses, courses)
        
        # Save the computed minimum courses for level j in the memo table.
        memo[j] = minCourses
        return minCourses
    
    # Start the helper function from level 1 to n.
    return helper(1, n)

In [27]:
## Test Code: Do not edit
print(minCoursesForJaneAvoidKK_Memoize(9)) # should be 2
print(minCoursesForJaneAvoidKK_Memoize(13)) # should be 2
print(minCoursesForJaneAvoidKK_Memoize(19)) # should be 4
print(minCoursesForJaneAvoidKK_Memoize(34)) # should be 5
print(minCoursesForJaneAvoidKK_Memoize(43)) # should be 5
print(minCoursesForJaneAvoidKK_Memoize(55)) # should be 6
print(minCoursesForJaneAvoidKK_Memoize(69)) # should be 8
print(minCoursesForJaneAvoidKK_Memoize(812)) # should be 83

2
2
4
5
5
6
8
83


## 2(C) Recover the solution in terms of number of jumps for each course.

In [28]:
def minCoursesForJaneAvoidKK_Solution(n):
    # Initialize the memoization table with None values for courses and an empty list for jumps.
    memo_courses = [float('inf')] * (n + 1)
    memo_jumps = [[] for _ in range(n + 1)]
    
    # Base case initialization
    memo_courses[1] = 0  # No courses needed to stay at level 1
    
    # Populate the memo tables
    for j in range(1, n + 1):
        for step in [1, 4, 5, 11]:
            next_level = j + step
            if next_level <= n and (next_level % 7 != 2 or next_level == n):
                if memo_courses[next_level] > memo_courses[j] + 1:
                    memo_courses[next_level] = memo_courses[j] + 1
                    memo_jumps[next_level] = memo_jumps[j] + [step]
    
    # The solution for n is now stored in memo_courses[n] and memo_jumps[n]
    return memo_courses[n], memo_jumps[n]

In [29]:
## Test Code: Do not edit
print(minCoursesForJaneAvoidKK_Solution(9)) # should be 2, [4, 4]
print(minCoursesForJaneAvoidKK_Solution(13)) # should be 2, [11, 1]
print(minCoursesForJaneAvoidKK_Solution(19)) # should be 4, [4, 5, 4, 5]
print(minCoursesForJaneAvoidKK_Solution(34)) # should be 5, [5, 1, 11, 11, 5]
print(minCoursesForJaneAvoidKK_Solution(43)) # should be 5, [4, 5, 11, 11, 11]
print(minCoursesForJaneAvoidKK_Solution(55)) # should be 6, [5, 11, 11, 11, 11, 5]
print(minCoursesForJaneAvoidKK_Solution(69)) # should be 8, [11, 1, 11, 11, 11, 11, 11, 1]
print(minCoursesForJaneAvoidKK_Solution(812)) # should be 83, [5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11]

(2, [4, 4])
(2, [11, 1])
(4, [5, 1, 1, 11])
(5, [5, 1, 11, 11, 5])
(5, [4, 5, 11, 11, 11])
(6, [5, 11, 11, 11, 11, 5])
(8, [11, 1, 11, 11, 11, 11, 11, 1])
(83, [5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11])


## Question 3: Energize Jane with a budget.

Unfortunately, life was not as simple as it seemed in problem 2. Besides dealing with Kilokahn, taking a course with a level jump consumes
a lot of Jane's energy, and she has an energy $E_0$ to begin with. Each time Jane jumps levels, she loses energy as follows:


| Jump   | Energy Consumed |
|--------|-----------------|
|  1     |       1         |
|  4     |       2         |
|  5     |       3         |
| 11     |       7         |


If at any point her energy level is $ \leq 0$ (even if she is at the destination), she will lose.

Given $n$, and initial energy $E_0$, plan how Jane can reach level $n$ (ninja level, in case you forgot) while
avoiding Kilokahn who  lurks when dividing the level by $7$ leaves a remainder of $2$ and keeping her energy levels
always strictly positive.

----

### 3(A): Write a Recurrence

Write a recurrence `minCoursesWithEnergyBudget(j, E, n)` that given that Jane is currently on level `j` with energy `E` finds the minimal 
number of courses she needs to take to reach `n`. Do not forget the base cases.

In [27]:
def minCoursesWithEnergyBudget(j, E, n):
    # Base case: If Jane's current level is greater than or equal to n, no more courses are needed.
    if j >= n:
        return 0
    
    # Base case: If energy level is less than or equal to 0, return infinity to indicate failure.
    if E <= 0:
        return float('inf')

    # Initialize minimum courses to a large number.
    minCourses = float('inf')  # Initialize with infinity, as we are looking for the minimum.
    
    # Energy consumption mapping for each jump.
    energy_consumption = {1: 1, 4: 2, 5: 3, 11: 7}

    # Iterate through possible steps.
    for step in [1, 4, 5, 11]:
        next_level = j + step
        # Update energy after making the jump.
        next_energy = E - energy_consumption.get(step, 0)

        # Check if the next level is safe (not leaving a remainder of 2 when divided by 7) or it's the target level.
        # Also, ensure that energy level is positive after making the jump.
        if (next_level % 7 != 2 or next_level == n) and next_level <= n and next_energy > 0:

            # Compute the minimum number of courses required if moving to the next level is safe and energy is sufficient.
            courses = 1 + minCoursesWithEnergyBudget(next_level, next_energy, n)
            minCourses = min(minCourses, courses)

    return minCourses

In [28]:
# test code do not edit
print(minCoursesWithEnergyBudget(1, 25, 10)) # must be 2
print(minCoursesWithEnergyBudget(1, 25, 6)) # must be 1
print(minCoursesWithEnergyBudget(1, 25, 30)) # must be 5
print(minCoursesWithEnergyBudget(1, 16, 30)) # must be 7
print(minCoursesWithEnergyBudget(1, 18, 31)) # must be 7
print(minCoursesWithEnergyBudget(1, 22, 38)) # must be 7
print(minCoursesWithEnergyBudget(1, 32, 55)) # must be 11
print(minCoursesWithEnergyBudget(1, 35, 60)) # must be 12

2
1
5
7
7
7
11
12


## 3(B): Memoize the Recurrence

Write a memo table to memoize the recurrence. Your memo table must be  of the form $T[j][e]$ for $j$ ranging from $1$ to $n$
and $e$ ranging from $0$ to $E$. You will have to handle the base cases carefully.

In [29]:
def minCoursesWithEnergyBudget_Memoize(E, n):
    # Energy consumption mapping for each jump.
    energy_consumption = {1: 1, 4: 2, 5: 3, 11: 7}

    # Initialize memoization table with None
    # Plus one to accommodate the range up to n and E inclusively
    T = [[None for _ in range(E + 1)] for _ in range(n + 1)]
    
    def dp(j, e):
        # Base cases
        if j >= n:  # If Jane's current level is at or beyond the target, no more courses are needed.
            return 0
        if e <= 0:  # If energy is zero or negative, this path fails.
            return float('inf')

        # Check if we have already computed this state
        if T[j][e] is not None:
            return T[j][e]

        # Initialize minimum courses for this state
        minCourses = float('inf')
        
        for step in [1, 4, 5, 11]:
            next_level = j + step
            next_energy = e - energy_consumption.get(step, 0)
            
            # Ensure next level and energy are valid and compute minimum courses if valid
            if (next_level % 7 != 2 or next_level == n) and next_level <= n and next_energy > 0:
                courses = 1 + dp(next_level, next_energy)
                minCourses = min(minCourses, courses)
        
        # Memoize and return the computed minimum
        T[j][e] = minCourses
        return minCourses

    # Call the dp function starting from level 1 with initial energy E
    return dp(1, E)

In [30]:
# test code do not edit
print(minCoursesWithEnergyBudget_Memoize(25, 10)) # must be 2
print(minCoursesWithEnergyBudget_Memoize(25, 6)) # must be 1
print(minCoursesWithEnergyBudget_Memoize(25, 30)) # must be 5
print(minCoursesWithEnergyBudget_Memoize(16, 30)) # must be 7
print(minCoursesWithEnergyBudget_Memoize(18, 31)) # must be 7
print(minCoursesWithEnergyBudget_Memoize(22, 38)) # must be 7
print(minCoursesWithEnergyBudget_Memoize(32, 55)) # must be 11
print(minCoursesWithEnergyBudget_Memoize(35, 60)) # must be 12

2
1
5
7
7
7
11
12


## 3(C): Recover the Solution

Now write code that will also return the minimum number of courses along with the list of jumps that will achieve this minimum number

In [33]:
def minCoursesWithEnergyBudget_Solution(E, n):
    # Energy consumption mapping for each jump.
    energy_consumption = {1: 1, 4: 2, 5: 3, 11: 7}

    # Initialize memoization table with None for minimum courses and an empty list for the path.
    T = [[None for _ in range(E + 1)] for _ in range(n + 1)]
    
    def dp(j, e):
        # Base case: If Jane's current level is at or beyond the target.
        if j >= n:
            return (0, [])
        # Base case: If energy is zero or negative, this path fails.
        if e <= 0:
            return (float('inf'), [])
        
        # Check if we have already computed this state.
        if T[j][e] is not None:
            return T[j][e]

        # Initialize minimum courses for this state with infinity and empty path.
        minCourses = (float('inf'), [])
        
        for step in [1, 4, 5, 11]:
            next_level = j + step
            next_energy = e - energy_consumption[step]
            
            # Ensure next level and energy are valid and compute minimum courses if valid.
            if (next_level % 7 != 2 or next_level == n) and next_level <= n and next_energy > 0:
                courses, path = dp(next_level, next_energy)
                # Update minimum if a new minimum is found.
                if courses + 1 < minCourses[0]:
                    minCourses = (courses + 1, [step] + path)  # Prepend step to path
        
        # Memoize and return the computed minimum along with the path.
        T[j][e] = minCourses
        return minCourses

    # Call the dp function starting from level 1 with initial energy E.
    minCourses, path = dp(1, E)
    return minCourses, path

In [34]:
# test code do not edit
print(minCoursesWithEnergyBudget_Solution(25, 10)) # must be 2, [4,5]
print(minCoursesWithEnergyBudget_Solution(25, 6)) # must be 1, [5]
print(minCoursesWithEnergyBudget_Solution(25, 30)) # must be 5, [4, 5, 4, 5, 11]
print(minCoursesWithEnergyBudget_Solution(16, 30)) # must be 7, [4, 5, 4, 4, 4, 4, 4]
print(minCoursesWithEnergyBudget_Solution(18, 31)) # must be 7, [4, 5, 4, 4, 4, 4, 5]
print(minCoursesWithEnergyBudget_Solution(22, 38)) # must be 7,  [4, 5, 4, 4, 4, 5, 11]
print(minCoursesWithEnergyBudget_Solution(32, 55)) # must be 11, [4, 5, 4, 4, 4, 4, 5, 4, 4, 11, 5]
print(minCoursesWithEnergyBudget_Solution(35, 60)) # must be 12, [4, 5, 4, 4, 4, 4, 5, 4, 4, 11, 5, 5]

(2, [4, 5])
(1, [5])
(5, [4, 5, 4, 5, 11])
(7, [4, 5, 4, 4, 4, 4, 4])
(7, [4, 5, 4, 4, 4, 4, 5])
(7, [4, 5, 4, 4, 4, 5, 11])
(11, [4, 5, 4, 4, 4, 4, 5, 4, 4, 11, 5])
(12, [4, 5, 4, 4, 4, 4, 5, 4, 4, 11, 5, 5])


----

## Question 4: Subset Sum Problem

We are given a set of whole numbers $S:\ \{ n_1, \ldots, n_k \}$ and a number $N$.
Our goal is to choose a subset of numbers $T:\ \{ n_{i_1}, \ldots, n_{i_j} \} \subseteq S$ such that

   (a) $\sum_{l=1}^j n_{i_l}  \leq N$, the sum of chosen numbers is less than or equal to $N$, 

   (b) The difference $N - \sum_{l=1}^j n_{i_l} $ is made as small as possible.

 For example, $S = \{ 1, 2, 3, 4, 5, 10 \}$ and $N = 20$ then by choosing $T = \{1, 2, 3, 4, 5\}$, we have  
$1 + 2 + 3 + 4 + 5 = 15 \leq 20$, achieving a difference of $5$. However, if we chose $T = \{ 2,3,5,10\}$ 
we obtain a sum of $2 + 3 + 5 + 10 = 20$ achieving the smallest possible difference of $0$.


Therefore the problem is as follows:

  * Inputs: list  $S: [n_1, \ldots, n_k]$ and number $N$.
  * Output: a list $T$ of elements from $S$ such that sum of elements of $T$ is  $\leq N$ and $N - \sum_{e \in T} e$ is the smallest possible.

The subsequent parts to this problem ask you to derive a dynamic programming solution to this problem.

__Note:__ Because $S$ and $T$ are viewed as sets, each element in the set may occur exactly once.

 ## 4(A) Show how the decisions can be staged to obtain optimal substructure (expected size: 5 lines)

Decision at each stage: At each stage, for a current number $n_i$ in set $S$, we decide whether to include $n_i$ in the subset $T$ or not. This decision is based on whether including $n_i$ helps us get closer to, but not exceed, the target sum $N$.

Optimal Substructure: The optimal solution to the problem for a set $S$ of size $k$ and target sum $N$ depends on the optimal solutions to smaller problems. Specifically, for each number $n_i$, we solve two subproblems: one where $n_i$ is included (if it does not cause the sum to exceed $N$) and one where $n_i$ is excluded. The better of these two solutions (i.e., the one that yields a sum closest to $N$) is chosen for each $n_i$.

Overlapping Subproblems: As we break down the problem into smaller subproblems, we find that the same subproblem (with the same remaining sum $N - \sum_{e \in T} e$ and the same subset of $S$ remaining to be considered) can arise multiple times. This redundancy is a key indicator that dynamic programming can be used to store intermediate results and avoid recomputing them, thus improving efficiency.

Bottom-Up Approach: We can construct a table where each cell $(i, j)$ represents the maximum sum we can achieve using the first $i$ numbers of $S$ without exceeding the sum $j$. We fill this table in a bottom-up manner, starting from the smallest subproblems (using no numbers to achieve sum $0$) up to the entire set $S$ to achieve sum $N$.

Reconstruction of Solution: After filling the table, we trace back from the cell representing the target sum $N$ using the entire set $S$ to determine which elements were included in the optimal subset $T$. This backtracking is possible because each entry in the table contains not just the value of the optimal solution up to that point but also implicitly which decision (include or exclude the current number) led to that optimal solution.

## 4(B): Write a recursive function for calculating the minimum value of the difference possible. 

In [35]:
def minSubsetDifference_recursive(N, s_list):
    def helper(index, current_sum):
        # Base case: If we've considered all elements
        if index == len(s_list):
            return abs(N - current_sum)
        
        # Include the current element in the subset
        include = helper(index + 1, current_sum + s_list[index])
        
        # Exclude the current element from the subset
        exclude = helper(index + 1, current_sum)
        
        # Return the minimum of including or excluding the current element
        return min(include, exclude)
    
    # Call the helper function starting with the first index and sum of 0
    return helper(0, 0)

In [36]:
# Code for testing your solution
# DO NOT EDIT
print(minSubsetDifference_recursive(15, [1, 2, 3, 4, 5, 10])) # Should be zero
print(minSubsetDifference_recursive(26, [1, 2, 3, 4, 5, 10])) # should be 1
print(minSubsetDifference_recursive(23, [1, 2, 3, 4, 5, 10])) # should be 0
print(minSubsetDifference_recursive(18, [1, 2, 3, 4, 5, 10])) # should be 0
print(minSubsetDifference_recursive(9, [1, 2, 3, 4, 5, 10])) # should be 0
print(minSubsetDifference_recursive(457, [11, 23, 37, 48, 94, 152, 230, 312, 339, 413])) # should be 1
print(minSubsetDifference_recursive(512, [11, 23, 37, 48, 94, 152, 230, 312, 339, 413])) # should be 0
print(minSubsetDifference_recursive(616, [11, 23, 37, 48, 94, 152, 230, 312, 339, 413])) # should be 1

0
1
0
0
0
1
0
1


## 4(C): Memoize the recurrence above. 

To help with your memoization, use a 2D memo table  $T[n][j]$ that represents the value for `minSubsetDifference(n, s_list[0:j])`. 

In [39]:
def minSubsetDifference_Memoize(N, s_list):
    # Create a memoization dictionary to store results of subproblems
    memo = {}
    
    # Nested helper function to implement the recursion with additional parameters
    def helper(index, current_sum):
        # Check if the result is already in the memo
        if (index, current_sum) in memo:
            return memo[(index, current_sum)]
        
        # Base case: If we've considered all elements
        if index == len(s_list):
            return abs(N - current_sum)
        
        # Include the current element in the subset
        include = helper(index + 1, current_sum + s_list[index])
        
        # Exclude the current element from the subset
        exclude = helper(index + 1, current_sum)
        
        # Store the result in memo before returning
        memo[(index, current_sum)] = min(include, exclude)
        
        return memo[(index, current_sum)]
    
    # Call the helper function starting with the first index and sum of 0
    return helper(0, 0)

In [40]:
# Code for testing your solution
# DO NOT EDIT
print(minSubsetDifference_Memoize(15, [1, 2, 3, 4, 5, 10])) # Should be 0
print(minSubsetDifference_Memoize(26, [1, 2, 3, 4, 5, 10])) # should be 1
print(minSubsetDifference_Memoize(23, [1, 2, 3, 4, 5, 10])) # should be 0
print(minSubsetDifference_Memoize(18, [1, 2, 3, 4, 5, 10])) # should be 0
print(minSubsetDifference_Memoize(9, [1, 2, 3, 4, 5, 10])) # should be 0
print(minSubsetDifference_Memoize(457, [11, 23, 37, 48, 94, 152, 230, 312, 339, 413])) # should be 1
print(minSubsetDifference_Memoize(512, [11, 23, 37, 48, 94, 152, 230, 312, 339, 413])) # should be 0
print(minSubsetDifference_Memoize(616, [11, 23, 37, 48, 94, 152, 230, 312, 339, 413])) # should be 1

0
1
0
0
0
1
0
1


## 4(D): Write code to recover the solution

In [55]:
def minSubsetDifference(N, s_list):
    memo = {}

    def helper(index, current_sum):
        # Check if the result is already in the memo
        if (index, current_sum) in memo:
            return memo[(index, current_sum)]
        if index == len(s_list):
            return abs(N - current_sum), []

        include_diff, include_subset = helper(index + 1, current_sum + s_list[index])
        exclude_diff, exclude_subset = helper(index + 1, current_sum)

        if include_diff < exclude_diff:
            result = (include_diff, include_subset + [s_list[index]])
        else:
            result = (exclude_diff, exclude_subset)

        memo[(index, current_sum)] = result
        return result

    min_diff, subset = helper(0, 0)
    return min_diff, subset

In [56]:
# Code for testing your solution
# DO NOT EDIT
print(minSubsetDifference(15, [1, 2, 3, 4, 5, 10])) # Should be 0, [5, 4, 3, 2, 1]
print(minSubsetDifference(26, [1, 2, 3, 4, 5, 10])) # should be 1, [10, 5, 4, 3, 2, 1]
print(minSubsetDifference(23, [1, 2, 3, 4, 5, 10])) # should be 0, [10, 5, 4, 3, 1]
print(minSubsetDifference(18, [1, 2, 3, 4, 5, 10])) # should be 0, [10, 4, 3, 1]
print(minSubsetDifference(9, [1, 2, 3, 4, 5, 10])) # should be 0, [4, 3, 2]
print(minSubsetDifference(457, [11, 23, 37, 48, 94, 152, 230, 312, 339, 413])) # should be 1, [339, 94, 23]
print(minSubsetDifference(512, [11, 23, 37, 48, 94, 152, 230, 312, 339, 413])) # should be 0, [312, 152, 37, 11]
print(minSubsetDifference(616, [11, 23, 37, 48, 94, 152, 230, 312, 339, 413])) # should be 1, [413, 94, 48, 37]

(0, [10, 5])
(1, [10, 5, 4, 3, 2, 1])
(0, [10, 5, 4, 3, 1])
(0, [10, 5, 3])
(0, [5, 4])
(1, [339, 94, 23])
(0, [312, 152, 48])
(1, [339, 230, 48])


### I gave up on trying to get the correct solution sorry.

## 4 (E): Greedy Solution

Suppose we use the following greedy solution to solve the problem.
  * $T = \emptyset$
  * While ( $ N \geq 0 $) 
    * Select the largest element $e$ for $S$ that is smaller than $N$
    * Remove $e$ from $S$
    * Add $e$ to $T$
    * N = N - e
  * return (N, T)
  
Using an example, show that the greedy algorithm does not necessarily produce the optimal solution.

### Answer (4 lines)

The greedy algorithm does not always produce the optimal solution because it makes the local optimal choice at each step with the hope of finding the global optimum without considering the overall problem.

For example, let's consider the set S={1,3,4,9} and N=10. Following the greedy algorithm:

We first select the largest element smaller than NN, which is 9, and subtract it from N, leaving N=1.

The remaining elements in S are {1,3,4}, and we select 1 because it's the only one left smaller than the new N.

This gives us T={9,1} and N=0.

However, the optimal solution is T={4,3,3}, which also results in N=0 but uses a smaller number of elements from S. This example shows that the greedy solution can fail to find the minimum difference or the most efficient subset of elements that achieves that difference, demonstrating it does not guarantee an optimal solution.

## Testing your solutions -- Do not edit code beyond this point